In [14]:
# Import libraries and packages for the project 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv
import requests

print('- Finish importing packages')

- Finish importing packages


In [17]:
DRIVER_PATH = '../driver/mac/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.maximize_window()

In [18]:
def login():
      sleep(2)
      url = 'https://www.linkedin.com/login'
      driver.get(url)
      print('- Finish initializing a driver')
      sleep(1)

      # Task 1.2: Import username and password
      credential = open('credentials.txt')
      line = credential.readlines()
      username = line[0]
      password = line[1]
      print('- Finish importing the login credentials')
      sleep(1)

      # Task 1.2: Key in login credentials
      email_field = driver.find_element_by_id('username')
      email_field.send_keys(username)
      print('- Finish keying in email')
      sleep(3)

      password_field = driver.find_element_by_name('session_password')
      password_field.send_keys(password)
      print('- Finish keying in password')
      # sleep(3)

      # Task 1.2: Click the Login button
      # signin_field = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
      # signin_field.click()
      # sleep(1.5)

      print('- Finish Task 1: Login to Linkedin')

      # Task 1.3: Check if the login is successful
login()




- Finish initializing a driver
- Finish importing the login credentials
- Finish keying in email
- Finish keying in password
- Finish Task 1: Login to Linkedin


In [13]:

def search(key=''):
    search_field = driver.find_element_by_class_name('search-global-typeahead__input')
    # Task 2.2: Input the search query to the search bar
    keyword = key + ' ' + 'people'
    search_field.send_keys(keyword)

    # Task 2.3: Search
    search_field.send_keys(Keys.RETURN)
    print('- Finish Task 2: Search for profiles')
    sleep(2)
    see_full_result = driver.find_element_by_css_selector('#main > div > div > div:nth-child(1) > div.search-results__cluster-bottom-banner.artdeco-button.artdeco-button--tertiary.artdeco-button--muted > a')
    see_full_result.click()
    
search('Tester')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=101.0.4951.64)


In [20]:
# Task 3: Scrape the URLs of the profiles

# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'app-aware-link')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        profile_URL = profile_ID.split('?')[0].split('/')[4]
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

# Task 3.2: Navigate through many page, and extract the profile URLs of each page

def FullURL():
    input_page = 4
    URLs_all_page = []
    for page in range(input_page):
        URLs_one_page = GetURL()
        # print(URLs_one_page)
        sleep(2)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
        sleep(3)
        # next_button = driver.find_element_by_class_name("artdeco-pagination__button--next")
        try:
            next_button = driver.find_element_by_css_selector('[aria-label=Next]')
        except:
            print("Error when Loading")
            driver.refresh()
            continue
            
        driver.execute_script("arguments[0].click();", next_button)
        URLs_all_page = URLs_all_page + URLs_one_page
        sleep(3)
    return URLs_all_page

print('- Finish Task 3: Scrape the URLs')

# URLs_all_page = FullURL()


# import pandas as pd 
# df = pd.DataFrame(URLs_all_page)
# print(df)
# # df.to_csv('profileURL_Tester.csv',mode='a', header=False)



- Finish Task 3: Scrape the URLs


In [111]:
def getUserProfile(id=''):
    driver.get('https://www.linkedin.com/in/' + id)
    name = ''
    role = ''
    name = driver.find_elements_by_class_name('text-heading-xlarge')[0].text
    # print("Name: ", name)
    # driver.get('https://www.linkedin.com/in/' + id +'/details/experience')
    # experience_list = driver.find_elements_by_class_name('optional-action-target-wrapper')
    # role = driver.find_element_by_xpath('/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[2]').text
    role = driver.find_element_by_css_selector('#ember37 > div.ph5 > div.mt2.relative > div:nth-child(1) > div.text-body-medium.break-words').text
    # print(role)
    sleep(0.5)
    skill_list = getSkills(id)
    # for u in experience_list:
    #     print(u.text)
    # Task 4: Scrape the profile information
    return name,role,skill_list


def getExperience(id=''):
    driver.get('https://www.linkedin.com/in/' + id +'/details/experience')
    experience = []
    page_source =  BeautifulSoup(driver.page_source,'html.parser')
    
    list_experience = page_source.select('div.display-flex.flex-row.justify-space-between')
    # print("LENGTH: ", len(list_experience))

    count = 0
    for item in list_experience:
        item_hidden = item.find_all('span',{'aria-hidden':'true'})
        count += 1
        # print("COUNT: ", count)
        exp = ''
        for i in item_hidden:
            exp = exp + i.text + ','
        exp = exp[:-1]
        experience.append(exp)
    skill_list = getSkills(id)
    return experience,skill_list

def getEducation(id=''):
    driver.get('https://www.linkedin.com/in/' + id +'/details/education')
    education = []
    page_source =  BeautifulSoup(driver.page_source,'html.parser')
    
    list_experience = page_source.select('div.display-flex.flex-row.justify-space-between')
    # print("LENGTH: ", len(list_experience))

    count = 0
    for item in list_experience:
        item_hidden = item.find_all('span',{'aria-hidden':'true'})
        count += 1
        # print("COUNT: ", count)
        exp = ''
        for i in item_hidden:
            exp = exp + i.text + ','
        exp = exp[:-1]
        education.append(exp)
    
    skill_list = getSkills(id)
    return education,skill_list


def getSkills(id=''):
    driver.get('https://www.linkedin.com/in/' + id +'/details/skills')
    # pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated 
    sleep(0.6)
    skill_list = []
    page_source =  BeautifulSoup(driver.page_source,'html.parser')
    list_skill = page_source.select('span.mr1.t-bold')
    # print("LENGTH: ", len(list_skill))
    
    for item in list_skill:
        # print(item)
        item_list = item.find_all('span',{'aria-hidden':'true'})
        
        # print(item_list)
        for i in item_list:
            if i.text not in skill_list:
                skill_list.append(i.text)
    # print(skill_list)
    return skill_list


name,role,skill_list=getUserProfile('cuong-nguyen-0a9379a8')
# getUserProfile('thanh-truc-do-30946a235')
# getUserProfile('jolly-nguyen-5a2558179')
# # sleep(0.5)
# exp = getExperience('jolly-nguyen-5a2558179')
# # sleep(0.5)
# exp,skills = getEducation('jolly-nguyen-5a2558179')
# # # # education
# sleep(4)
# skills = getSkills('cuong-nguyen-0a9379a8')
print(name,role)
# print(exp)
# print(education)
print(skill_list)

Cuong Nguyen Data Interoperability Engineer at PATH
['R', 'Python', 'MySQL', 'Microsoft SQL Server', 'Statistical Data Analysis', 'SQL Server Integration Services (SSIS)', 'SQL Server Reporting Services (SSRS)', 'Tableau', 'Power BI', 'Data Analysis', 'Business Intelligence', 'Data Mining', 'Business Analytics', 'Regression Analysis', 'Cluster Analysis', 'Data Classification', 'Data Cleaning', 'Principal Component Analysis', 'Factor Analysis', 'Machine Learning', 'HTML Scripting', 'Data Integration', 'Extract, Transform, Load (ETL)', 'Logistic Regression', 'International Financial Reporting Standards (IFRS)', 'Transfer Pricing Analysis', 'Market Analysis', 'Customer Analysis', 'PHP', 'Cascading Style Sheets (CSS)', 'Microsoft Excel', 'MongoDB', 'SAS', 'Python (Programming Language)', 'Java', 'Management', 'Analytical Skills']


In [ ]:
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file

with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Accessing profile: ', linkedin_URL)

        page_source = bs4(driver.page_source, "html.parser")

        info_div = page_source.find('div',{'class':'flex-1 mr5'})
        info_loc = info_div.find_all('ul')
        name = info_loc[0].find('li').get_text().strip() #Remove unnecessary characters 
        print('--- Profile name is: ', name)
        location = info_loc[1].find('li').get_text().strip() #Remove unnecessary characters 
        print('--- Profile location is: ', location)
        title = info_div.find('h2').get_text().strip()
        print('--- Profile title is: ', title)
        writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
        print('\n')

print('Mission Completed!')

Mission Completed!


In [ ]:
linkedin_URL ='https://www.linkedin.com/in/caroline-almaraz-942476168/'
driver.get(linkedin_URL)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

profile_experience = []
profile_education = []
profile_skills = []
profile_certification = []
profile_honors = []


# profile_name = soup.find_all('h1',{'class':'text-heading-xlarge inline t-24 v-align-middle break-words'})[0].get_text().strip()

profile_experience = soup.find_all('section',{'class':'artdeco-card'})
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
for i in range(0,len(profile_experience)):
      header_tag = profile_experience[i].select('h2.pvs-header__title.text-heading-large>span.visually-hidden')
      if len(header_tag) > 0:
            if(header_tag[0].get_text().strip() == 'Experience'):
                  experience_list = profile_experience[i].select('div.display-flex.flex-row.justify-space-between')
                  print(experience_list)           
      

